In [1]:
import sys
sys.path.insert(0, 'src')

# Force reimport
if 'pixel_planet.vertex_ai_agent' in sys.modules:
    del sys.modules['pixel_planet.vertex_ai_agent']
if 'pixel_planet.spatial_utils' in sys.modules:
    del sys.modules['pixel_planet.spatial_utils']
if 'pixel_planet.config' in sys.modules:
    del sys.modules['pixel_planet.config']

from pixel_planet.vertex_ai_agent import VertexAIAgent
import inspect

# Verify the fix is in the code
source = inspect.getsource(VertexAIAgent)
if 'QueryJobConfig(use_legacy_sql=False)' in source:
    print("✅ Fix is loaded in memory")
else:
    print("❌ Fix not loaded - need to restart kernel")

❌ Fix not loaded - need to restart kernel


In [1]:
# Cell 1: Load environment variables (handles export and quotes)
import os
from pathlib import Path

env_file = Path('.env')
if env_file.exists():
    with open(env_file) as f:
        for line in f:
            line = line.strip()
            # Skip comments and empty lines
            if not line or line.startswith('#'):
                continue
            
            # Remove 'export ' prefix if present
            if line.startswith('export '):
                line = line[7:]  # Remove 'export '
            
            # Split on first =
            if '=' in line:
                key, value = line.split('=', 1)
                # Remove quotes from value
                value = value.strip().strip('"').strip("'")
                os.environ[key] = value
    
    print("✓ Loaded .env file")
    print(f"  GCP_PROJECT_ID: {os.environ.get('GCP_PROJECT_ID', 'NOT SET')}")
    print(f"  BQ_DATASET: {os.environ.get('BQ_DATASET', 'NOT SET')}")
    print(f"  BQ_TABLE: {os.environ.get('BQ_TABLE', 'NOT SET')}")
else:
    print("⚠️  No .env file found")

✓ Loaded .env file
  GCP_PROJECT_ID: pixel-planet-101
  BQ_DATASET: weather
  BQ_TABLE: forecast_results


In [2]:
# After restarting kernel, run:
from pixel_planet.vertex_ai_agent import VertexAIAgent

agent = VertexAIAgent()

result = agent.assess_activity(
    location_name="Mt. Apo",
    latitude=6.987,
    longitude=125.273,
    start_time="2025-10-04T05:00:00",
    end_time="2025-10-05T21:00:00",
    activity_type="hiking"
)

# Check if chart_data is present
if 'chart_data' in result:
    print("✅ Chart data available!")
    print(f"Parameters: {list(result['chart_data']['forecasts'].keys())}")
else:
    print("❌ Chart data missing")

/Users/kyle/Desktop/pixel-planet-101/.venv/lib/python3.11/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/kyle/Desktop/pixel-planet-101/.venv/lib/python3.11/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
/Users/kyle/Desktop/pixel-planet-101/.venv/lib/python3.11/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credenti

✓ Agent initialized
  Project: pixel-planet-101
  Model: gemini-2.0-flash-exp
  Region: us-central1

🏃 Activity Safety Assessment
  Location: Mt. Apo (6.987, 125.273)
  Activity: hiking
  Time: 2025-10-04T05:00:00 to 2025-10-05T21:00:00


❓ Question: 
Assess the safety and suitability for hiking at Mt. Apo 
(coordinates: latitude 6.987, longitude 125.273) 
from 2025-10-04T05:00:00 to 2025-10-05T21:00:00.

Please:
1. Query the weather forecast for this location and time period
2. Analyze all weather parameters against safety thresholds for hiking
3. Determine if conditions are suitable or unsuitable
4. If unsuitable, recommend specific alternative time slots
5. Return a complete JSON response with:
   - Safety assessment
   - All 6 weather parameters with full time series data
   - Alternative time recommendations
   - Location and confidence information

Return your response as a valid JSON object following the specified format.
Include ALL forecast data for charting purposes.



E0000 00:00:1759598195.398176 10005514 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



💡 Answer:
```json
{
  "assessment": {
    "suitable": false,
    "risk_level": "High",
    "confidence": "High",
    "primary_concerns": "High temperatures pose a significant risk of heat exhaustion. Strong winds at higher elevations could also be dangerous.",
    "recommendation": "Hiking is not recommended during the specified period due to high temperatures and strong winds. Consider alternative times with cooler temperatures and lighter winds, such as early morning or late afternoon."
  },
  "forecast_summary": {
    "precipitation": {
      "min": 0.0,
      "max": 2.1,
      "avg": 0.1,
      "extreme_hours": []
    },
    "temperature": {
      "min": 16.2,
      "max": 36.3,
      "avg": 26.8,
      "extreme_hours": [
        "2025-10-04T11:00:00",
        "2025-10-04T12:00:00",
        "2025-10-04T13:00:00",
        "2025-10-04T14:00:00",
        "2025-10-04T15:00:00",
        "2025-10-05T11:00:00",
        "2025-10-05T12:00:00",
        "2025-10-05T13:00:00",
        "2025-1

In [3]:
print("AI Assessment:")
print(f"  Suitable: {result['assessment']['suitable']}")
print(f"  Risk: {result['assessment']['risk_level']}")
print(f"  Recommendation: {result['assessment']['recommendation']}")


AI Assessment:
  Suitable: False
  Risk: high
  Recommendation: Hiking is not recommended during the specified period due to high solar radiation, high temperatures, and the risk of heavy precipitation. Consider alternative times with lower temperatures, less intense sunlight, and minimal rainfall.


In [4]:
import pandas as pd
import matplotlib.pyplot as plt

chart_data = result['chart_data']

# Create a chart for each parameter
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
fig.suptitle(f"Weather Forecast for {chart_data['location']['name']}", fontsize=16)

parameters = ['temperature', 'precipitation', 'wind', 'humidity', 'solar_radiation', 'cloud_cover']
for idx, param in enumerate(parameters):
    ax = axes[idx // 2, idx % 2]
    
    data = chart_data['forecasts'].get(param, [])
    if data:
        df = pd.DataFrame(data)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Plot main line
        ax.plot(df['timestamp'], df['value'], label=param.replace('_', ' ').title(), linewidth=2)
        
        # Add confidence intervals if available
        if 'lower' in df.columns and 'upper' in df.columns:
            ax.fill_between(df['timestamp'], df['lower'], df['upper'], alpha=0.2)
        
        ax.set_xlabel('Time')
        ax.set_ylabel('Value')
        ax.set_title(param.replace('_', ' ').title())
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# 3. Summary statistics
print("\nWeather Summary:")
for param, stats in chart_data['summary_stats'].items():
    print(f"  {param}: min={stats['min']}, max={stats['max']}, avg={stats['avg']:.2f}")

# 4. Location info
loc = chart_data['location']
print(f"\nLocation Info:")
print(f"  Interpolation: {loc['interpolation_used']}")
print(f"  Nearest distance: {loc['nearest_distance_km']}km")
print(f"  Confidence: {loc['confidence']}")

KeyError: 'chart_data'

In [3]:
import json
import pprint
pprint.pprint(result, indent=2, width=120, compact=False)

{ 'alternative_times': [ { 'end': '2025-10-04T21:00:00',
                           'reason': 'Lower temperatures and decreasing solar radiation in the late afternoon.',
                           'start': '2025-10-04T17:00:00'},
                         { 'end': '2025-10-05T08:00:00',
                           'reason': 'Early morning offers cooler temperatures and lower solar radiation before the '
                                     'heat of the day sets in.',
                           'start': '2025-10-05T05:00:00'},
                         { 'end': '2025-10-06T08:00:00',
                           'reason': 'The next day has similar early morning conditions with cooler temperatures.',
                           'start': '2025-10-06T05:00:00'}],
  'assessment': { 'confidence': 'High',
                  'primary_concerns': 'High solar radiation and high temperatures pose significant risks of heat '
                                      'exhaustion and sunburn during the hike.',


In [2]:
# Cell 2: Initialize agent
import sys
sys.path.insert(0, 'src')

from pixel_planet.vertex_ai_agent import VertexAIAgent

agent = VertexAIAgent()

# Cell 3: Ask question
response = agent.ask(
    "What is the forecasted precipitation and temperature for coordinates "
    "latitude 7, longitude 125 on 2025-10-01? Include the confidence intervals."
)

print("\n✅ Test complete!")

/Users/kyle/Desktop/pixel-planet-101/.venv/lib/python3.11/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/kyle/Desktop/pixel-planet-101/.venv/lib/python3.11/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
/Users/kyle/Desktop/pixel-planet-101/.venv/lib/python3.11/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credenti

✓ Agent initialized
  Project: pixel-planet-101
  Model: gemini-2.0-flash-exp
  Region: us-central1

❓ Question: What is the forecasted precipitation and temperature for coordinates latitude 7, longitude 125 on 2025-10-01? Include the confidence intervals.


E0000 00:00:1759594391.412104 9877708 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



🤖 Agent iteration 1...
   Calling: query_bigquery

🔍 Executing query...
   Reason: Retrieve forecasted precipitation and temperature, including confidence intervals, for the specified coordinates and date.
   SQL: SELECT forecast_timestamp, parameter, forecast_value, prediction_interval_lower, prediction_interval_upper FROM `pixel-planet-101.weather.forecast_results` WHERE lat = 7 AND lon = 125 AND forecast_dat...
   ✓ Query returned 0 rows

💡 Answer:
There is no data available for the specified coordinates and date. I am unable to provide the forecasted precipitation and temperature. I would recommend checking if the date is within the available range, and that the coordinates are valid.


✅ Test complete!
